---
참고 url : https://wikidocs.net/3683

##### 단일데이터 가져오기 코든데 뭐에 사용하는지는 잘 모르겠음
---

---
##### 필요한 import , login

In [3]:
import win32com.client
import pythoncom

class XASessionEventHandler:
    login_state = 0

    def OnLogin(self, code, msg):
        if code == "0000":
            print("로그인 성공")
            XASessionEventHandler.login_state = 1
        else:
            print("로그인 실패")

class XAQueryEventHandlerT1102:
    query_state = 0

    def OnReceiveData(self, code):
        XAQueryEventHandlerT1102.query_state = 1


In [4]:
# ----------------------------------------------------------------------------
# login
# ----------------------------------------------------------------------------
id = "ssuyan"
passwd = "tndus1!!"
cert_passwd = "qkrtndus1!!"

instXASession = win32com.client.DispatchWithEvents("XA_Session.XASession", XASessionEventHandler)
instXASession.ConnectServer("hts.ebestsec.co.kr", 20001)
instXASession.Login(id, passwd, cert_passwd, 0, 0)

while XASessionEventHandler.login_state == 0:
    pythoncom.PumpWaitingMessages()


로그인 성공


---
##### TR 코드에 대한 이번트 처리할 클래스이다.
##### query_state라는 이름의 클래스 변수를 사용해 이벤트 처리 여부 저장
    
    - 요청한 작업이 완료되면 query_state 값이 1로 변경된다.

In [5]:
class XAQueryEventHandlerT1102:
    query_state = 0

    def OnReceiveData(self, code):
        XAQueryEventHandlerT1102.query_state = 1

In [6]:
instXAQueryT1102 = win32com.client.DispatchWithEvents("XA_DataSet.XAQuery", XAQueryEventHandlerT1102)

In [7]:
instXAQueryT1102.ResFileName = "C:\\eBEST\\xingAPI\\Res\\t1102.res"

---
##### 
1. 파이썬 코드로 입력 데이터를 설정하려면 XAQuery 인스턴스를 통해 SetFieldData라는 메서드를 호출한 후 적절한 인자 값을 지정


2. SetFieldData의 1번째 파라미터는 블록명이고, 2번째 파라미터는 필드명입니다. 3번째 파라미터에는 단일 데이터를 조회할 때는 0을 지정하면 되고, 4번째 파라미터는 필드에 해당하는 입력값이다.

In [8]:
instXAQueryT1102.SetFieldData("t1102InBlock", "shcode", 0, "078020")

In [9]:
instXAQueryT1102.Request(0)

17

In [10]:
while XAQueryEventHandlerT1102.query_state == 0:
    pythoncom.PumpWaitingMessages()

---
- t1102OutBlock 블록명과 원하는 필드명을 입력해 데이터를 가져올 수 있습니다.    


    - 예를 들어, "hname" 필드명을 지정하면 한글종목명을 가져올 수 있고, "price" 필드명을 지정하면 종목의 현재가를 가져올 수 있습니다.

In [11]:
name = instXAQueryT1102.GetFieldData("t1102OutBlock", "hname", 0)
price = instXAQueryT1102.GetFieldData("t1102OutBlock", "price", 0)
print(name)
print(price)

이베스트투자증권
4495


---
##### 특정기간 데이터 가져오기
---

In [13]:
import win32com.client
import pythoncom

class XASessionEventHandler:
    login_state = 0

    def OnLogin(self, code, msg):
        if code == "0000":
            print("로그인 성공")
            XASessionEventHandler.login_state = 1
        else:
            print("로그인 실패")

class XAQueryEventHandlerT8413:
    query_state = 0

    def OnReceiveData(self, code):
        XAQueryEventHandlerT8413.query_state = 1


In [15]:
# ----------------------------------------------------------------------------
# login
# ----------------------------------------------------------------------------
id = "ssuyan"
passwd = "tndus1!!"
cert_passwd = "qkrtndus1!!"

instXASession = win32com.client.DispatchWithEvents("XA_Session.XASession", XASessionEventHandler)
instXASession.ConnectServer("hts.ebestsec.co.kr", 20001)
instXASession.Login(id, passwd, cert_passwd, 0, 0)

while XASessionEventHandler.login_state == 0:
    pythoncom.PumpWaitingMessages()

로그인 성공


In [16]:

# ----------------------------------------------------------------------------
# T8413
# ----------------------------------------------------------------------------
instXAQueryT8413 = win32com.client.DispatchWithEvents("XA_DataSet.XAQuery", XAQueryEventHandlerT8413)
instXAQueryT8413.ResFileName = "C:\\eBEST\\xingAPI\\Res\\t8413.res"

instXAQueryT8413.SetFieldData("t8413InBlock", "shcode", 0, "078020")
instXAQueryT8413.SetFieldData("t8413InBlock", "gubun", 0, "2")
instXAQueryT8413.SetFieldData("t8413InBlock", "sdate", 0, "20160111")
instXAQueryT8413.SetFieldData("t8413InBlock", "edate", 0, "20160122")
instXAQueryT8413.SetFieldData("t8413InBlock", "comp_yn", 0, "N")

instXAQueryT8413.Request(0)


40

In [17]:
while XAQueryEventHandlerT8413.query_state == 0:
    pythoncom.PumpWaitingMessages()

In [18]:
count = instXAQueryT8413.GetBlockCount("t8413OutBlock1")
for i in range(count):
    date = instXAQueryT8413.GetFieldData("t8413OutBlock1", "date", i)
    open = instXAQueryT8413.GetFieldData("t8413OutBlock1", "open", i)
    high = instXAQueryT8413.GetFieldData("t8413OutBlock1", "high", i)
    low = instXAQueryT8413.GetFieldData("t8413OutBlock1", "low", i)
    close = instXAQueryT8413.GetFieldData("t8413OutBlock1", "close", i)
    print(date, open, high, low, close)

20160111 9810 9860 9700 9800
20160112 9670 9990 9670 9750
20160113 9790 9840 9790 9800
20160114 9560 9810 9560 9690
20160115 9820 9820 9590 9590
20160118 9590 9590 9450 9450
20160119 9270 9320 9240 9240
20160120 9570 9570 9020 9080
20160121 9100 9260 9050 9050
20160122 9020 9200 8880 9060
